# Personal Pools

Launch this tutorial in a Jupyter Notebook on Binder: 
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/htcondor/htcondor-python-bindings-tutorials/master?urlpath=lab/tree/Personal-Pools.ipynb)

A Personal HTCondor Pool is an HTCondor Pool that has a single owner, who is:
- The pool’s administrator.
- The only submitter who is allowed to submit jobs to the pool.
- The owner of all resources managed by the pool.

The HTCondor Python bindings provide a submodule, `htcondor.personal`, which allows you to manage personal pools from Python.
Personal pools are useful for:
- Utilizing local computational resources (i.e., all of the cores on a lab server).
- Created an isolated testing/development environment for HTCondor workflows.
- Serving as an entrypoint to other computational resources, like annexes or flocked pools (not yet implemented).

We can start a personal pool by instantiating a `PersonalPool`.
This object represents the personal pool and lets us manage its "lifecycle": start up and shut down.
We can also use the `PersonalPool` to interact with the HTCondor pool once it has been started up.

Each Personal Pool must have a unique "local directory", corresponding to the HTCondor configuration parameter `LOCAL_DIR`. For this tutorial, we'll put it in the current working directory so that it's easy to find.

> Advanced users can configure the personal pool using the `PersonalPool` constructor. See the documentation for details on the available options.

In [1]:
import htcondor
from htcondor.personal import PersonalPool
from pathlib import Path

In [2]:
pool = PersonalPool(local_dir = Path.cwd() / "personal-condor")
pool

PersonalPool(local_dir=./personal-condor, state=INITIALIZED)

To tell the personal pool to start running, call the `start()` method:

In [3]:
pool.start()

PersonalPool(local_dir=./personal-condor, state=READY)

`start()` doesn't return until the personal pool is `READY`, which means that it can accept commands (e.g., job submission).

`Schedd` and `Collector` objects for the personal pool are available as properties on the `PersonalPool`:

In [4]:
pool.schedd

In [5]:
pool.collector

For example, we can submit jobs using `pool.schedd`:

In [6]:
sub = htcondor.Submit(
    executable = "/bin/sleep",
    arguments = "$(ProcID)s",
)

schedd = pool.schedd
submit_result = schedd.submit(sub, count=10)

print(f"ClusterID is {submit_result.cluster()}")

ClusterID is 2


And we can query for the state of those jobs:

In [7]:
for ad in pool.schedd.query(
    constraint = f"ClusterID == {submit_result.cluster()}", 
    projection = ["ClusterID", "ProcID", "JobStatus"]
):
    print(repr(ad))

[ ProcID = 0; ClusterID = 2; JobStatus = 1; ServerTime = 1695159761 ]
[ ProcID = 1; ClusterID = 2; JobStatus = 1; ServerTime = 1695159761 ]
[ ProcID = 2; ClusterID = 2; JobStatus = 1; ServerTime = 1695159761 ]
[ ProcID = 3; ClusterID = 2; JobStatus = 1; ServerTime = 1695159761 ]
[ ProcID = 4; ClusterID = 2; JobStatus = 1; ServerTime = 1695159761 ]
[ ProcID = 5; ClusterID = 2; JobStatus = 1; ServerTime = 1695159761 ]
[ ProcID = 6; ClusterID = 2; JobStatus = 1; ServerTime = 1695159761 ]
[ ProcID = 7; ClusterID = 2; JobStatus = 1; ServerTime = 1695159761 ]
[ ProcID = 8; ClusterID = 2; JobStatus = 1; ServerTime = 1695159761 ]
[ ProcID = 9; ClusterID = 2; JobStatus = 1; ServerTime = 1695159761 ]


We can use the collector to query the state of pool:

In [8]:
# get 3 random ads from the daemons in the pool
for ad in pool.collector.query()[:3]:
    print(ad)


    [
        AuthenticatedIdentity = "condor@family"; 
        EffectiveQuota = 0.0; 
        Priority = 5.000000000000000E+02; 
        Requested = 0.0; 
        UpdateSequenceNumber = 3; 
        PriorityFactor = 1.000000000000000E+03; 
        AuthenticationMethod = "FAMILY"; 
        AccountingGroup = "<none>"; 
        Name = "<none>"; 
        SubtreeQuota = 0.0; 
        IsAccountingGroup = true; 
        MyType = "Accounting"; 
        NegotiatorName = "jovyan@fa6c829ace67"; 
        GroupSortKey = 0.0; 
        ResourcesUsed = 0; 
        ConfigQuota = 0.0; 
        DaemonStartTime = 1695159756; 
        BeginUsageTime = 0; 
        LastHeardFrom = 1695159760; 
        WeightedAccumulatedUsage = 0.0; 
        AccumulatedUsage = 0.0; 
        TargetType = "none"; 
        WeightedResourcesUsed = 0.0; 
        DaemonLastReconfigTime = 1695159756; 
        SurplusPolicy = "byquota"; 
        LastUsageTime = 0; 
        LastUpdate = 1695159760
    ]

    [
        UpdatesLost_Co

When you're done using the personal pool, you can `stop()` it:

In [9]:
pool.stop()

PersonalPool(local_dir=./personal-condor, state=STOPPED)

`stop()`, like `start()` will not return until the personal pool has actually stopped running.
The personal pool will also automatically be stopped if the `PersonalPool` object is garbage-collected, or when the Python interpreter stops running.

> To prevent the pool from being automatically stopped in these situations, call the `detach()` method. The corresponding `attach()` method can be used to "re-connect" to a detached personal pool.

When working with a personal pool in a script, you may want to use it as a context manager. This pool will automatically start and stop at the beginning and end of the context:

In [10]:
with PersonalPool(local_dir = Path.cwd() / "another-personal-condor") as pool:  # note: no need to call start()
    print(pool.get_config_val("LOCAL_DIR"))

/home/jovyan/tutorials/another-personal-condor
